In [1]:
import numpy as np
import pickle
import math
import os
import matplotlib.pyplot as plt
from utils_sage import get_cableidx
import collections
###################
nth_case_list=np.arange(1,440+1)
n_f1=6
n_f2=7
n_f=12
n_node=106
###################

###cable data
#a=[3463.605,3078.76,2693.915,2309.07,1924.225,1539.38,1154.535,769.69,384.845,0]
a=[9,8,7,6,5,4,3,2,1,0]
cable_list=[]#[[3848.45]*40] #440-idx,cable_no,[40] #k=0
for e in range(1,41):
    for j in a:
        #k=k+1;print(k-1,e-1,j)#filename, elementidx, area-float
        c=[10]*40#c=[3848.45]*40
        c[e-1]=j
        #cable_list.append([k-1,e-1,c])
        cable_list.append(c)
len(cable_list)
###node ddata - included nodes
node_cable= get_cableidx()#e.copy()
in_cable=list(range(3,40,4))#[3,7,11,15,19,23,27,31,35,39]
node_cable1=node_cable[:,1]-1
node_cable2=node_cable[:,2]-1
# out_cable=set(list(range(40)))- set(in_cable);out_cable=list(out_cable)
# out_node=np.append(node_cable1[out_cable],node_cable2[out_cable]); out_node=list(set(out_node))
# in_node=set(list(range(106)))-set(out_node);in_node=list(in_node)
in_node=np.append(node_cable1[in_cable],node_cable2[in_cable]).tolist()
in_node.sort()
print(in_node,len(in_node))
out_node=list(set(range(106))-set(in_node))
print(out_node,len(out_node))
#node ddata - node signal
with open("data_sage/node_signal_list.pkl", 'rb') as pickle_file:
    _node_signal_list = pickle.load(pickle_file)
    
_node_signal_list=_node_signal_list[1:]#########classifcation

#node data
x_list=[]
y_list=[]
for i in range(len(_node_signal_list)):
    node_signal=_node_signal_list[i]
    cable=cable_list[i]
    data = np.concatenate([node_signal], axis=-1)
    x_list.append(data)
    y_list.append([cable]*len(data))
x=np.concatenate(x_list,axis=0)
y=np.concatenate(y_list,axis=0)
print(x.shape,y.shape)#(26387, 106, 12) (26387, 40)#(15418, 12, 106, 7) (15418, 12, 106, 7)*106),dtype='int8')

########################################
# time_step=4
# x_offsets = np.sort(np.concatenate((np.arange(0, time_step, 1),)))
# ########################################
# x_list=[]
# y_list=[]
# for i in range(len(_node_signal_list)):
#     node_signal=_node_signal_list[i]
#     cable=cable_list[i]
#     data = np.concatenate([node_signal], axis=-1)
#     x_t=[]#(s,12,106,12)
#     for t in range(0,len(data)-time_step+1):
#         x_t.append(data[t + x_offsets, ...])
#     #print(len(x_t),np.array(x_t).shape)
#     x_t=np.array(x_t).reshape((-1,n_node*time_step,n_f))
#     x_list.append(x_t)
#     y_list.append([cable]*len(x_t))
# x=np.concatenate(x_list,axis=0)
# y=np.concatenate(y_list,axis=0)
# print(x.shape,y.shape)#(22056, 1272, 12) (22056, 40)

Using backend: pytorch


[6, 14, 20, 28, 36, 45, 53, 59, 67, 75, 81, 82, 88, 89, 94, 97, 98, 101, 104, 105] 20
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 83, 84, 85, 86, 87, 90, 91, 92, 93, 95, 96, 99, 100, 102, 103] 86
(26387, 106, 12) (26387, 40)


In [2]:
########delete 86 nodes#######
N=len(x)
# x=np.concatenate([x,np.zeros((N,n_node,1))],axis=2)#add empty node feature
# x[:,in_node,:]=1
# n_f=x.shape[2]#13
#############################

#scale-features
features=x.copy()
for i in range(n_f):
    #mini=np.min(features[...,i].flatten());maxi=np.max(features[...,i].flatten());print(mini,maxi)
    #features[...,i]=(features[...,i]-mini)/(maxi-mini)
    m=np.mean(features[...,i].flatten());std=np.std(features[...,i].flatten())
    features[...,i]=(features[...,i]-m)/std
    print(i,np.min(features[...,i]),np.max(features[...,i]),np.mean(features[...,i]),np.std(features[...,i]))
print(features.shape)
#scale-label
print(np.mean(y.flatten()),np.std(y.flatten()))
#label=(y-np.mean(y.flatten()))/np.std(y.flatten())
label=y.copy()
print(features.shape,label.shape)#(149, 106, 6) (149, 40) (185, 2) (185, 2)

#train/val/test split
indices = np.random.permutation(len(features))
n_training=int(len(features)*0.8)
n_val=int(len(features)*0.1)
n_test=len(features)-n_training-n_val
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
f_training, f_val, f_test = features[training_idx],features[val_idx], features[test_idx]
l_training, l_val, l_test = label[training_idx],label[val_idx], label[test_idx]
print(f_training.shape,f_val.shape,f_test.shape,l_training.shape,l_val.shape,l_test.shape)#(21173, 106, 6) (2646, 106, 6) (2648, 106, 6) (21173, 40) (2646, 40) (2648, 40)

0 -1.743677 23.5342 -2.3115765e-08 0.9999999
1 -48.85317 51.611217 -8.438093e-09 1.0000002
2 -35.922775 42.27882 9.00136e-10 1.0
3 -57.401787 30.076107 1.8471882e-08 1.0
4 -77.90262 51.637733 3.6851022e-09 1.0000002
5 -38.00022 110.11309 -2.7720097e-09 1.0
6 -8.738964 5.914165 9.359232e-08 0.9999998
7 -27.905714 27.91171 -6.991602e-08 0.99999976
8 -5.412336 1.3196024 -3.0517338e-08 1.0000001
9 -9.597588 9.592666 -8.9031635e-09 1.0
10 -4.4488354 4.673789 1.0916195e-08 1.0000001
11 -26.082317 26.082935 1.1347169e-09 0.9999999
(26387, 106, 12)
9.86547068632281 0.9528649553387615
(26387, 106, 12) (26387, 40)
(21109, 106, 12) (2638, 106, 12) (2640, 106, 12) (21109, 40) (2638, 40) (2640, 40)


In [3]:
#mask
mask=np.array([False]*106);mask[node_cable1]=True
mask_traini=np.array([mask]*n_training);mask_vali=np.array([mask]*n_val);mask_testi=np.array([mask]*n_test)#np.concatenate()
mask_train=np.concatenate([mask_traini,np.zeros_like(mask_vali, dtype=bool),np.zeros_like(mask_testi, dtype=bool)],axis=0)
mask_val=np.concatenate([np.zeros_like(mask_traini, dtype=bool),mask_vali,np.zeros_like(mask_testi, dtype=bool)],axis=0)
mask_test=np.concatenate([np.zeros_like(mask_traini, dtype=bool),np.zeros_like(mask_vali, dtype=bool),mask_testi],axis=0)
mask_train=mask_train.reshape((-1,1));mask_val=mask_val.reshape((-1,1));mask_test=mask_test.reshape((-1,1))
mask_train.shape,mask_val.shape,mask_test.shape#((2797022, 1), (2797022, 1), (2797022, 1))

#features
features=np.concatenate([f_training.reshape((-1,n_f)),f_val.reshape((-1,n_f)),f_test.reshape((-1,n_f))])
print(features.shape)#(2797022, 12)

#labels
'''
#labeling for graph classification
label_train=np.argmax((l_training!=l_training[0][0]),axis=1)
label_val=np.argmax((l_val!=l_training[0][0]),axis=1)
label_test=np.argmax((l_test!=l_training[0][0]),axis=1)
labels=np.concatenate([label_train,label_val,label_test],axis=0)'''
label_train=np.zeros((n_training,106));label_train[:,node_cable1]=l_training
label_val=np.zeros((n_val,106));label_val[:,node_cable1]=l_val
label_test=np.zeros((n_test,106));label_test[:,node_cable1]=l_test
labels=np.concatenate([label_train.reshape((-1,1)),label_val.reshape((-1,1)),label_test.reshape((-1,1))],axis=0)
print(labels.shape)#(2797022, 1)
print(collections.Counter(labels.flatten()))

(2797022, 12)
(2797022, 1)


In [5]:
data_path='data_sage/'

np.savez_compressed(data_path+'features',features)
np.savez_compressed(data_path+'labels',labels)
np.savez_compressed(data_path+'mask_train',mask_train)
np.savez_compressed(data_path+'mask_val',mask_val)
np.savez_compressed(data_path+'mask_test',mask_test)

In [48]:
import numpy as np
import pickle
import collections
adj_mx=np.load('data_sage/sensor_graph/adj_wself.npy')
edge_data=[[i,j] for i,j in zip(np.where(adj_mx==1)[0],np.where(adj_mx==1)[1])];edge_data=np.array(edge_data)
edge_data.shape#(476, 2)
#multiple graphy adj mx
adj=edge_data.copy();N=26387
adj=np.concatenate([adj]*N)#adj.shape(12560212, 2)
n_edge=edge_data.shape[0]#476
gidx=[[i]*n_edge for i in range(0,106*(N),106)]
gidx=np.array(gidx)
gidx=gidx.flatten()
gidx=np.stack([gidx,gidx],axis=1)
adj=adj+gidx#(12560212, 2)

(476, 2)

In [64]:
np.savez_compressed('data_sage/edge_data',adj)